In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import numpy as np
import pandas as pd
from pandas_datareader import wb


In [2]:

# Indicator mapping
indicators = {
    'NY.GDP.PCAP.CD': 'GDP_per_capita',
    'NY.GDP.MKTP.KD.ZG': 'GDP_growth',
    'FP.CPI.TOTL.ZG': 'Inflation',
    'GC.DOD.TOTL.GD.ZS': 'Debt_to_GDP',
    'BN.CAB.XOKA.GD.ZS': 'Current_account_balance',
    'SL.UEM.TOTL.ZS': 'Unemployment',
    'PV.EST': 'Political_stability'
}

# Fetch from World Bank
try:
    wb_data = wb.download(
        indicator=list(indicators.keys()),
        country='all',
        start=2000,
        end=2023
    ).reset_index()

    # Rename columns
    wb_data = wb_data.rename(columns=indicators)

    # Pivot to wide format
    wb_data = wb_data.pivot_table(
        index=['country', 'year'],
        values=list(indicators.values()),
        aggfunc='first'
    ).reset_index()

    # Save to disk
    wb_data.to_csv('../data/raw/world_bank_indicators.csv', index=False)
    print("Saved World Bank indicators to '../data/raw/world_bank_indicators.csv'")

except Exception as e:
    print(f"Error: {e}")


Saved World Bank indicators to '../data/raw/world_bank_indicators.csv'


In [3]:

# Load and inspect first few rows
wb = pd.read_csv('../data/raw/world_bank_indicators.csv')
print(wb.shape)
wb.head()


(6280, 9)


,country,year,Current_account_balance,Debt_to_GDP,GDP_growth,GDP_per_capita,Inflation,Political_stability,Unemployment
0,Afghanistan,2000,NaN,NaN,NaN,174.930991,NaN,-2.438969,7.935
1,Afghanistan,2001,NaN,NaN,-9.431974,138.706822,NaN,NaN,7.953
2,Afghanistan,2002,NaN,NaN,28.600001,178.954088,NaN,-2.035034,7.930
3,Afghanistan,2003,NaN,NaN,8.832278,198.871116,NaN,-2.198372,7.880
4,Afghanistan,2004,NaN,NaN,1.414118,221.763654,NaN,-2.295682,7.899


In [14]:
# === Define paths (relative to notebooks folder) ===
ratings_path = '../data/raw/20241115 Moody\'s Investors Service Sovereign.csv'
defaults_path = '../data/raw/BoC-BoE-Database-2024.xlsx'
indicators_path = '../data/raw/world_bank_indicators.csv'

# === Load datasets ===

# Moody's ratings
ratings = pd.read_csv(ratings_path)

# BoC–BoE defaults (skip metadata rows)
defaults = pd.read_excel(defaults_path, sheet_name="Data_2024.csv", skiprows=64)

# World Bank indicators
indicators = pd.read_csv(indicators_path)

# === Show dataset shapes ===
print(f"Ratings shape:    {ratings.shape}")
print(f"Defaults shape:   {defaults.shape}")
print(f"Indicators shape: {indicators.shape}")

# === Preview data ===
display(ratings.head())
display(defaults.head())
display(indicators.head())



C:\Users\HP\AppData\Local\Temp\ipykernel_16256\2972722037.py:9: DtypeWarning: Columns (4,18,24) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings = pd.read_csv(ratings_path)


Ratings shape:    (138107, 32)
Defaults shape:   (10688, 49)
Indicators shape: (6280, 9)


,rating_agency_name,file_creating_date,sec_category,issuer_name,legal_entity_identifier,object_type_rated,instrument_name,CUSIP_number,coupon_date,maturity_date,...,issuer_identifier_schema,instrument_identifier,instrument_identifier_schema,central_index_key,obligor_identifier,obligor_identifier_schema,obligor_identifier_other,obligor_sec_category,obligor_industry_group,obligor_name
0,Moody's Investors Service,2024-11-15,NaN,NaN,549300MCOOY1V7P2PG30,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2296,NRSRO,NaN,Sovereign,NaN,Eutelsat SA
1,Moody's Investors Service,2024-11-15,NaN,NaN,549300MCOOY1V7P2PG30,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2296,NRSRO,NaN,Sovereign,NaN,Eutelsat SA
2,Moody's Investors Service,2024-11-15,NaN,NaN,549300MCOOY1V7P2PG30,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2296,NRSRO,NaN,Sovereign,NaN,Eutelsat SA
3,Moody's Investors Service,2024-11-15,NaN,NaN,549300MCOOY1V7P2PG30,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2296,NRSRO,NaN,Sovereign,NaN,Eutelsat SA
4,Moody's Investors Service,2024-11-15,NaN,NaN,549300MCOOY1V7P2PG30,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2296,NRSRO,NaN,Sovereign,NaN,Eutelsat SA


,k,DEBT_COUNTRY,DEBT_COUNTRY_GROUP,DEBT_YEAR,DEBT_TOTAL_2023,DEBT_IMF_2024,DEBT_IBRD_2024,DEBT_IDA_2024,DEBT_IADB_2024,DEBT_PARIS_CLUB_2024,...,DEBT_FC_LOANS_DEF_SOVEREIGNS_2024,DEBT_FC_BONDS_DEF_SOVEREIGNS_2024,DEBT_PRIVATE_CREDITORS_DEF_SOVEREIGNS_2024,DEBT_LC_DEBT_DEF_SOVEREIGNS_2024,DEBT_ARREARS_TOTAL_SOVEREIGNS_2024,DEBT_PARIS_CLUB_DEBT_2024,DEBT_GROSS_CHINA_2024,DEBT_GROSS_WORLD_PUBLIC_DEBT_2024,DEBT_GROSS_EME_OTHER_GDP_2024,DEBT_GROSS_WORLD_PRODUCT_2024
0,WORLD_1960,World,World,1960,13353.27,173,NaN,NaN,NaN,NaN,...,1.0,11.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1365.0
1,WORLD_1961,World,World,1961,15269.62,53,NaN,NaN,NaN,300,...,3.0,12.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,1420.0
2,WORLD_1962,World,World,1962,14661.35,54,NaN,NaN,NaN,270,...,1.0,11.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1525.0
3,WORLD_1963,World,World,1963,15053.83,25.3,NaN,NaN,NaN,NaN,...,2.0,11.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,1638.0
4,WORLD_1964,World,World,1964,16915.69,211,NaN,NaN,NaN,270,...,2.0,11.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,1800.0


,country,year,Current_account_balance,Debt_to_GDP,GDP_growth,GDP_per_capita,Inflation,Political_stability,Unemployment
0,Afghanistan,2000,NaN,NaN,NaN,174.930991,NaN,-2.438969,7.935
1,Afghanistan,2001,NaN,NaN,-9.431974,138.706822,NaN,NaN,7.953
2,Afghanistan,2002,NaN,NaN,28.600001,178.954088,NaN,-2.035034,7.930
3,Afghanistan,2003,NaN,NaN,8.832278,198.871116,NaN,-2.198372,7.880
4,Afghanistan,2004,NaN,NaN,1.414118,221.763654,NaN,-2.295682,7.899


In [5]:
# === Clean Moody's Sovereign Ratings ===

# Filter sovereign ratings
sovereign = ratings[ratings['obligor_sec_category'] == 'Sovereign'].copy()

# Drop rows with missing rating or country name
sovereign = sovereign.dropna(subset=['rating', 'obligor_name'])

# Extract year from file_creating_date
sovereign['year'] = pd.to_datetime(sovereign['rating_action_date'], errors='coerce').dt.year
sovereign = sovereign.dropna(subset=['year'])
sovereign['year'] = sovereign['year'].astype(int)

# remove withdrawn ratings (e.g., 'WR')
sovereign = sovereign[~sovereign['rating'].isin(['WR'])]

# Select and rename columns
sovereign_clean = sovereign[['obligor_name', 'year', 'rating']].rename(columns={'obligor_name': 'country'})
print(sovereign_clean.shape)
sovereign_clean.head()


(83258, 3)


,country,year,rating
0,Eutelsat SA,2012,Baa2
1,Eutelsat SA,2012,Baa2
2,Eutelsat SA,2013,Baa3
3,Eutelsat SA,2013,Baa3
6,Eutelsat SA,2018,Baa3


In [ ]:
from sentence_transformers import SentenceTransformer, util


# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')  # lightweight and fast

# Get country lists
moody_names = sovereign_clean['country'].unique()
reference_names = indicators['country'].unique()

# Compute embeddings
moody_embeds = model.encode(moody_names, convert_to_tensor=True)
reference_embeds = model.encode(reference_names, convert_to_tensor=True)

# Match using cosine similarity
auto_country_map = {}
for i, name in enumerate(moody_names):
    cosine_scores = util.cos_sim(moody_embeds[i], reference_embeds)[0]
    best_idx = int(np.argmax(cosine_scores))
    best_score = float(cosine_scores[best_idx])

    if best_score >= 0.85:  # Adjust threshold as needed
        auto_country_map[name] = reference_names[best_idx]

# Apply the mapping
sovereign_clean['country'] = sovereign_clean['country'].replace(auto_country_map)

# Optional: show sample mappings
print("Sample NLP-based auto-mappings:")
for k, v in list(auto_country_map.items())[:10]:
    print(f"{k} -> {v}")


Sample NLP-based auto-mappings:
Hong Kong SAR, China, Government of -> Hong Kong SAR, China
Kuwait, Government of -> Kuwait
Luxembourg, Government of -> Luxembourg
Malaysia, Government of -> Malaysia
Panama, Government of -> Panama
Saudi Arabia, Government of -> Saudi Arabia
Turkiye, Government of -> Turkiye
Cayman Islands, Government of -> Cayman Islands
Lebanon, Government of -> Lebanon
Mauritius, Government of -> Mauritius


In [15]:
# Ensure year is integer type
indicators['year'] = pd.to_numeric(indicators['year'], errors='coerce').astype('Int64')
indicators = indicators.dropna(subset=['year'])
indicators['year'] = indicators['year'].astype(int)

print("Indicators cleaned:", indicators.shape)
display(indicators.head())


Indicators cleaned: (6280, 9)


,country,year,Current_account_balance,Debt_to_GDP,GDP_growth,GDP_per_capita,Inflation,Political_stability,Unemployment
0,Afghanistan,2000,NaN,NaN,NaN,174.930991,NaN,-2.438969,7.935
1,Afghanistan,2001,NaN,NaN,-9.431974,138.706822,NaN,NaN,7.953
2,Afghanistan,2002,NaN,NaN,28.600001,178.954088,NaN,-2.035034,7.930
3,Afghanistan,2003,NaN,NaN,8.832278,198.871116,NaN,-2.198372,7.880
4,Afghanistan,2004,NaN,NaN,1.414118,221.763654,NaN,-2.295682,7.899


In [17]:
print(defaults.columns.tolist())


['k', 'DEBT_COUNTRY', 'DEBT_COUNTRY_GROUP', 'DEBT_YEAR', 'DEBT_TOTAL_2023', 'DEBT_IMF_2024', 'DEBT_IBRD_2024', 'DEBT_IDA_2024', 'DEBT_IADB_2024', 'DEBT_PARIS_CLUB_2024', 'DEBT_CHINA_2024', 'DEBT_OTHER_OFFICIAL_CREDITORS_2024', 'DEBT_PRIVATE_CREDITORS_2024', 'DEBT_FC_BANK_LOANS_2024', 'DEBT_FC_BONDS_2024', 'DEBT_LC_DEBT_2024', 'DEBT_FISCAL_ARREARS_2024', 'DEBT_TOTAL_DEBT_2024', 'DEBT_ADVANCED_ECON_2024', 'DEBT_EME_2024', 'DEBT_HIPC_2024', 'DEBT_OTHER_2024', 'DEBT_EXAE_2024', 'DEBT_SSA_2024', 'DEBT_DEFRATE_SOVEREIGNS_2024', 'DEBT_DEFRATE_PARIS_CLUB_DEBT_2024', 'DEBT_DEFRATE_CHINA_2024', 'DEBT_DEFRATE_WORLD_PUBLIC_DEBT_2024', 'DEBT_DEFRATE_EME_OTHER_GDP_2024', 'DEBT_DEFRATE_WORLD_GDP_2024', 'DEBT_TOTAL_NUMB_SOVEREIGNS_2024', 'DEBT_TOTAL_DEF_SOVEREIGNS_2024', 'DEBT_IMF_DEF_SOVEREIGNS_2024', 'DEBT_IBRD_DEF_SOVEREIGNS_2024', 'DEBT_IDA_DEF_SOVEREIGNS_2024', 'DEBT_IADB_DEF_SOVEREIGNS_2024', 'DEBT_PARIS_CLUB_DEF_SOVEREIGNS_2024', 'DEBT_CHINA_DEF_SOVEREIGNS_2024', 'DEBT_OTHER_OFFICIAL_CREDITORS_

In [18]:
# Update these names based on actual column names
defaults_clean = defaults[['DEBT_COUNTRY', 'DEBT_CO_DEBT_YEAR']].copy()
defaults_clean.columns = ['country', 'year']

# Assume that appearance of country in a given year = 1 default event
defaults_clean['default_flag'] = 1

# Remove rows with missing country or year
defaults_clean = defaults_clean.dropna(subset=['country', 'year'])

# Convert year to int
defaults_clean['year'] = defaults_clean['year'].astype(int)

# === Normalize country names using sentence-transformers ===
defaults_names = defaults_clean['country'].unique()
defaults_embeds = model.encode(defaults_names, convert_to_tensor=True)

defaults_map = {}
for i, name in enumerate(defaults_names):
    cos_scores = util.cos_sim(defaults_embeds[i], reference_embeds)[0]
    best_idx = int(np.argmax(cos_scores))
    best_score = float(cos_scores[best_idx])
    if best_score >= 0.85:
        defaults_map[name] = reference_names[best_idx]

# Apply mapping
defaults_clean['country'] = defaults_clean['country'].replace(defaults_map)

# Optional: Show sample mappings
print("Sample defaults mappings:")
for k, v in list(defaults_map.items())[:10]:
    print(f"{k} → {v}")


KeyError: "['DEBT_CO_DEBT_YEAR'] not in index"

In [16]:
# Keep only relevant columns
defaults_clean = defaults[['country_name', 'year', 'default_flag']].copy()
defaults_clean.columns = ['country', 'year', 'default_flag']

# Remove rows with missing country or year
defaults_clean = defaults_clean.dropna(subset=['country', 'year'])

# Convert year to int
defaults_clean['year'] = defaults_clean['year'].astype(int)

# Normalize country names with NLP
defaults_names = defaults_clean['country'].unique()
defaults_embeds = model.encode(defaults_names, convert_to_tensor=True)

defaults_map = {}
for i, name in enumerate(defaults_names):
    cos_scores = util.cos_sim(defaults_embeds[i], reference_embeds)[0]
    best_idx = int(np.argmax(cos_scores))
    best_score = float(cos_scores[best_idx])
    if best_score >= 0.85:
        defaults_map[name] = reference_names[best_idx]

# Apply mapping
defaults_clean['country'] = defaults_clean['country'].replace(defaults_map)

print("Sample defaults mappings:")
for k, v in list(defaults_map.items())[:10]:
    print(f"{k} -> {v}")


KeyError: "None of [Index(['country_name', 'year', 'default_flag'], dtype='object')] are in the [columns]"

In [ ]:
pd.read_excel(defaults_path, sheet_name="Data_2024.csv", nrows=10)




['OBSERVATIONS', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48']
